# **Big Data Analysis**

## **Pré-tratamento**

***Nesta secção irá ser feito o pré-tratamento dos dados***. Concretamente iremos ***fundir os dados*** provenientes dos diferentes datasets, *"alinhando-os"* pelas colunas *"Entidade"* (equivalente a nome do país), *"Code"* (equivalente ao código identificador IUPAC do país) e *"Year"* (correspondente ao ano em que foram recolhidos os dados).

Iremos também *"aparar"* os dados em função do indicador limitante, neste caso o ano de dados disponíveis, sendo o ***período de 1990 a 2016*** o período comum a todos os datasets.

**Para efeitos de comparação** iremos fazer merge dos datasets usando o package Pandas e PySpark, de modo a comparar a eficiência de ambos em diferentes contextos

### Pré-tratamento utilizando Pandas

In [1]:
# Com pandas
import pandas as pd

# Importamos o módulo time para efeitos de comparação de tempos de execução
import time

pd_start = time.time() # inicio do contador pandas

# Carregamos o dataset com dados populacionais
import pandas as pd
pop = pd.read_csv('datasets/population.csv')
pop = pop[pop['Year'] >= 1990]
pop = pop[pop['Year'] <= 2016]

# Carregamos o dataset para prevelência de obesidade
obes = pd.read_csv('datasets/share-of-adults-defined-as-obese.csv')
obes = obes[obes['Year'] >= 1990]

# Carregamos o dataset para prevalência doenças mentais
mental = pd.read_csv('datasets/mental-illnesses-prevalence.csv')
mental = mental[mental['Year'] <= 2016]

merged = pd.DataFrame()
merged = pd.merge(pop,obes,on=['Entity','Code','Year'])
merged = pd.merge(merged,mental,on=['Entity','Code','Year'])

pd_end = time.time() # fim do contador pandas

pd_elapsed = pd_end - pd_start # tempo de execução do contador pandas


# Salvar o ficheiro com os datasets juntos na diretoria indicada
merged.to_csv('datasets/merged_pandas.csv', index=False) 

merged.head()

,Entity,Code,Year,Population (historical estimates),"Indicator:Prevalence of obesity among adults, BMI &GreaterEqual; 30 (crude estimate) (%) - Sex:Both sexes",Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized,Depressive disorders (share of population) - Sex: Both - Age: Age-standardized,Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized,Bipolar disorders (share of population) - Sex: Both - Age: Age-standardized,Eating disorders (share of population) - Sex: Both - Age: Age-standardized
0,Afghanistan,AFG,1990,10694804,1.0,0.223206,4.996118,4.713314,0.703023,0.127700
1,Afghanistan,AFG,1991,10745168,1.1,0.222454,4.989290,4.702100,0.702069,0.123256
2,Afghanistan,AFG,1992,12057436,1.2,0.221751,4.981346,4.683743,0.700792,0.118844
3,Afghanistan,AFG,1993,14003764,1.2,0.220987,4.976958,4.673549,0.700087,0.115089
4,Afghanistan,AFG,1994,15455560,1.3,0.220183,4.977782,4.670810,0.699898,0.111815


### Pré-tratamento de dados utilizando Spark

In [ ]:
#pip install pyspark
# remover # para instalar o package pyspark antes dos imports

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

import time

# Criação de uma sessão Spark
Spark = SparkSession.builder.appName("BigDataAnalysis").getOrCreate()

#o tempo está a contar desde o inicio da execução da tarefa e não antes do início da sessão, mesmo assim Pysparks revelou ser mais lento neste caso
spark_start = time.time() 

# Carregar e aparar datasets

# Dataset da população
spark_pop = Spark.read.csv(
    'datasets/population.csv',
    header = True ,
    inferSchema = True
    )

spark_pop = spark_pop.filter(col('Year').between(1990, 2016))

# Carregamos o dataset para prevelência de obesidade
spark_obes = Spark.read.csv(
    'datasets/share-of-adults-defined-as-obese.csv',
    header = True,
    inferSchema = True
    )

spark_obes = spark_obes.filter(col('Year') >= 1990)

# Carregamos o dataset para prevalência doenças mentais
spark_mental = Spark.read.csv(
    'datasets/mental-illnesses-prevalence.csv',
    header = True,
    inferSchema = True
    )

spark_mental = spark_mental.filter(col('Year') <= 2016)

# Unimos os datasets com o mesmo critério (país, IUPAC, código)

spark_merged = spark_pop.join \
  (spark_obes, on = ['Entity','Code','Year'], how = 'inner').join \
  (spark_mental, on = ['Entity', 'Code','Year'], how = 'inner')

# Descomentar para ver o dataset
#spark_merged.show()

Spark.stop()

spark_end = time.time() # fim do contador spark

spark_elapsed = spark_end - spark_start # tempo de execução do contador spark

In [4]:
# spark_merged.write.format("csv").option("header", "true").mode("append").save('datasets/merged_spark.csv')
# não estou a conseguir salvar o ficheiro tratado com pyspark dá um erro estranho, relacionado com a diretoria, 
# ja tentei com "append" e "overwrite", deve ser mesmo da diretoria

In [5]:
# Comparamos o tempo de computação para cada um dos modelos

print(f'Com pandas, demorou {round(pd_elapsed, 3)} segundos',
      f'Com spark, demorou {round(spark_elapsed, 3)} segundos',
      sep = '\n')

Com pandas, demorou 0.189 segundos
Com spark, demorou 16.675 segundos


### Análise do pré-tratamento
Neste caso concreto percebemos que várias ferramentas têm comportamentos diferentes em função dos dados, o que é algo a ter em conta no contexto do que pretendemos fazer.

Numa próxima abordagem iremos perceber que em função do que podemos fazer há ferramentas/abordagens mais ou menos eficientes. Neste caso concreto, a utilização de Spark demonstrou-se como menos eficiente.

## Escolha da base de dados

**Tarefa:**  
Ver nas bases de dados qual se adequa melhor ao nosso caso:  

- Cassandra
- Hadoop base
- MongoDB

**Características do nosso dataset:**  
- O dataset parece ser bem estruturado com um esquema fixo.  
- Tem um tamanho moderado com 5.185 entradas.  
- Inclui dados históricos e estatísticas de saúde, o que pode implicar em consultas analíticas e relatórios.

**Cassandra:** 
É altamente escalável e projetado para lidar com grandes volumes de dados distribuídos por vários servidores.  
É otimizado para gravações rápidas e pode lidar com grandes volumes de dados com desempenho consistente.  
No entanto, pode não ser o ideal para consultas complexas ad-hoc, pois seu modelo de dados é otimizado para consultas conhecidas e repetitivas.  
  
**HBase:** 
Também é projetado para escalabilidade horizontal e pode lidar com grandes volumes de dados.  
É otimizado para leituras e gravações aleatórias em grandes datasets.  
Suporta consultas em tempo real em Big Data, mas pode ser complexo de configurar e gerenciar.  
  
**MongoDB:** 
É um banco de dados orientado a documentos, projetado para flexibilidade e escalabilidade.  
Suporta um esquema dinâmico, o que o torna uma boa escolha para dados que podem evoluir com o tempo.  
É conhecido por sua facilidade de uso, eficiência em consultas complexas e boas capacidades de agregação de dados.  
Tem um forte suporte para consultas ad-hoc, indexação e agregação de dados.


**Em resumo**, se a flexibilidade no schema dos dados, a facilidade de desenvolvimento, e análises complexas são importantes, MongoDB pode ser a melhor escolha. Se a escalabilidade e a distribuição geográfica são prioridades, consideremos o Cassandra. E se estamos a operar dentro de um ecossistema Hadoop e precisa de acesso rápido e em tempo real aos dados, HBase pode ser a opção mais adequada.

**Optei pela base de dados no mongoDB**  # espero nao termos limitações por isto...  

Username : bigdata  
pass : bioinfo24

In [2]:
# mongoimport  
# Imports content from an Extended JSON, CSV, or TSV export. Descomentar e substituir no codigo pela nossa BD

# mongoimport --uri  mongodb+srv://bigdata:<enter_password>@bigdataanalysis.e19vio9.mongodb.net/<enter_database_name>  --collection  <enter_collection_name>  --type  <enter JSON/CSV/TSV>   --file  <enter path/to/file>

In [3]:

# mongoexport
# Produces a JSON or CSV export of data stored in a MongoDB instance. Descomentar e substituir no codigo pela nossa BD

# mongoexport --uri  mongodb+srv://bigdata:<enter_password>@bigdataanalysis.e19vio9.mongodb.net/<enter_database_name>  --collection  <enter_collection_name>  --type  <enter JSON/CSV/TSV>  --out  <enter_file_name> 